# Python-Code paketieren

(vgl. [Packaging Python Projects](https://packaging.python.org/tutorials/packaging-projects/))

Wenn wir eine Python-Anwendung mit mehreren Modulen oder auch nur ein einzelnes Skript mit Abhängigkeiten erstellt haben, möchten wir diese in ein Paket zusammenbinden, damit wir diese Funktionalität leicht auf anderen Systemen installieren können.

Beispiel: [surveyresponse.py](surveyresponse.py)

Dieses Paket benötigt unter Linux keine Abhängigkeiten, auf Windows jedoch `pywin32`.

Ein Python-Paket hat üblicherweise folgende Struktur (Beispiel Survey Response-Applikation):
```
survey_response/
├── README.md
├── setup.cfg
├── pyproject.toml
├── survey_response.py
└── test/
    └── test_survey_response.py
```
Im obersten Verzeichnis liegt die Wurzel des (Git-) Repositories. Es enthält ein Unterverzeichnis mit dem gleichen Namen sowie die Files *setup.cfg* und *pyproject.toml*.
In *README.md* sollte Zweck und eine kurze Anleitung angegeben werden.

Das gleichnamige Unterverzeichnis ist das eigentliche Python-Modul. In diesem Beispiel liegt der Code im File *survey_response.py* (single file Modul).

Das File *setup.cfg* enthält Projekt-Metadaten, welche zur Paketierung benötigt werden (ausführlichere Anleitung auf der Seite von [setuptools](https://setuptools.pypa.io/en/latest/userguide/declarative_config.html)).

**pyproject.toml** ([doc](https://pip.pypa.io/en/stable/reference/build-system/pyproject-toml/))
* deklariert welches build system verwendet wird (hier: setuptools)
* enthält konfiguration von code quality tools wie pylint, tox, formatting

```toml
[build-system]
requires = ["setuptools >= 40.6.0", "wheel"]
build-backend = "setuptools.build_meta"
```

Dieses File soll in Zukunft `setup.cfg` ablösen. Experimentell kann [bereits jetzt](https://setuptools.pypa.io/en/latest/userguide/pyproject_config.html) der Inhalt von `setup.cfg` in `pyproject.toml` eingebettet werden. Im Gegensatz zu `setup.cfg` welches spezifisch für [setuptools](https://setuptools.pypa.io) ist, ist `pyproject.toml` universell und für jede Paketierungslösung gültig.
Die prominenteste Alternative als Paketierungs-Tool ist [Poetry](https://python-poetry.org/).

**setup.cfg** (Paket-Metainformation / Abhängigkeiten / Entry-Points / ...)

```ini
[metadata]
name = survey_response
version = 0.0.1
description = Tool to collect mail survey responses in a folder in a csv file.
long_description = file: README.md
long_description_content_type = text/markdown
url = https://github.com/eth-its/Python-for-SysAdmins/
author = Chuck Norris
author_email = python.sysadmins@id.ethz.ch
license = MIT

[options]
zip_safe = False
packages = find:
install_requires =
    pywin32; platform_system=='Windows'
```

- **name**: Der *distribution name* der des Pakets. Er muss eindeutig sein. Dies kann z.B. erreicht werden, wenn ein beliebiger Name mit dem Usernamen ergänzt wird.
- **version**: Version des Pakets.
- **description**: Kurze, einzeilige Zusammenfassung des Pakets.
- **long_description**: Kann via `file:` den Inhalt von `README.md` einlesen und als ausführliche Version von **description** verwenden.
- **long_description_content_type**: Gibt hier markdown als Inhaltstyp von `README.md` an
- **packages**: Liste aller importierten Python-Pakete, welche im Paket aufgenommen werden sollen. Diese Liste kann entweder manuell oder mit `find:` automatisiert erzeugt werden.
- **install_requires**: Abhängigkeiten zu externen Python packages. Hier wird mit `platform_system=='Windows'` angegeben, dass die Abhängigkeit nur unter Windows benötigt wird.

Ist die Paketstruktur mit Code und Setup-Konfiguration bereit, kann das Paket erstellt werden. Zu diesem Zweck muss im Wurzelverzeichnis des Pakets folgender Befehl aufgerufen werden.:

```sh
pip install .
```

Damit ist das Python-Paket lokal installiert. Dies kann mit `pip list` überprüft werden.

Mit folgenden Befehlen kann ein Archiv des Python-Pakets erzeugt werden:

```sh
pip install --upgrade build
python -m build
```
Mit dem ersten Befehl wird sichergestellt, dass die aktuellsten Versionen von *build* installiert ist. Mit dem zweiten Befehl wird das Paket als *tar*-File und als *wheel* verpackt und im Unterverzeichnis *dist* abgelegt.

Dieses File kann nun beispielsweise auf den Python Paket-Index (*PyPI*) [https://pypi.org/](https://pypi.org/) geladen werden. Dieser Upload auf *PyPI* erfolgt am besten mit Hilfe von [twine](https://twine.readthedocs.io/en/latest/).

## Versionsnummern

Die Versionsnummer eines Pakets, welches auf *PyPI* geladen werden soll, muss sich von einem existierenden Paket (mit dem gleichen Namen) unterscheiden. Sinnvollerweies wird die Versionsnummer im Format gemäss
[*Semantic Versioning*](https://semver.org/) angegeben: MAJOR.MINOR.PATCH.

Vor einem wichtigen Release (z.B. *1.0.0*) müssen unter Umständen sog. Alpha- und Beta-Release veröffentlicht werden. Eine gültie Versionshistorie kann wie folgt aussehen: `0.9.99` -> `1.0.0-alpha.1` -> `1.0.0-alpha.2` -> `1.0.0-beta.1` -> `1.0.0`.

## Kommandozeile

Wird eine Funktionalität, welche mit Python implementier worden ist, häufig gebraucht, ist es umständlich, immer `python my_func.py` eingeben zu müssen. Einfacher wäre es, nur `my_func` einzugeben.

Das kann mit dem *options.entry_points* in *setup.cfg* erreicht werden:
```ini
[options.entry_points]
console_scripts =
    mail_dir2csv=survey_response:main
```
Mit dem `entry_points` in `console_scripts` wird der Setup so konfiguriert, dass ein Kommandozeilen-Programm mit dem Namen *mail_dir2csv* erzeugt wird. Wird dieses gestartet, so wird die `main`-Funktion im Modul *survey_response.main* ausgeführt.

Weiterführende Informationen auf [setuptools.pypa.io](https://setuptools.pypa.io/en/latest/userguide/entry_point.html).

## Multi file Modules

Im obigen Beispiel haben wir nur ein einzelnes Skript [surveyresponse.py](surveyresponse.py) paketiert.
Wenn das Prokekt wächst und das Skript in mehrere Files gegliedert wird, verändert sich die Ordnerstruktur wie folgt:
```
survey_response/
├── README.md
├── setup.cfg
├── pyproject.toml
├── survey_response
|   ├── __init__.py
|   ├── __main__.py
|   ├── file_module_a.py
|   ├── folder_module_b
|   |   ├── __init__.py
|   |   ├── file_sub_module.py
|   |   └── ...
|   └── ...
|
└── test/
    └── test_survey_response.py
```

`options.entry_points` muss lediglich zu
```ini
[options.entry_points]
console_scripts =
    mail_dir2csv=survey_response.__main__:main
```
verändert werden.